In [7]:
import glob
import re
import ast
import os
import networkx as nx
from collections import defaultdict

def read_adjacency_from_file(path):
    with open(path, 'r') as f:
        content = f.read()
    match = re.search(r"Room Adjacency List:\s*(\{.*\})", content)
    if not match:
        raise ValueError(f"Could not find 'Room Adjacency List' in {path}.")
    return ast.literal_eval(match.group(1))

def build_graph(adjacency_dict):
    G = nx.Graph()
    for room, neighbors in adjacency_dict.items():
        for nbr, w in neighbors.items():
            G.add_edge(room, nbr, weight=w)
    return G

def largest_connected_subgraph(G):
    if nx.is_connected(G):
        return G
    largest_cc = max(nx.connected_components(G), key=len)
    return G.subgraph(largest_cc).copy()

def compute_indices(G):
    V = G.number_of_nodes()
    E = G.number_of_edges()
    EI = V + E
    MI = (E - V + 1) / (2.0 * V - 5) if (2 * V - 5) > 0 else 0.0
    H = largest_connected_subgraph(G)
    ASPLI = nx.average_shortest_path_length(H, weight='weight')
    return EI, MI, ASPLI

def compute_difficulty(ei, mi, aspli,
                       ei_min, ei_max,
                       mi_min, mi_max,
                       aspli_min, aspli_max):
    def safe_norm(x, xmin, xmax):
        return 0.0 if xmax == xmin else (x - xmin) / (xmax - xmin)
    EI_n    = safe_norm(ei,    ei_min,    ei_max)
    MI_n    = safe_norm(mi,    mi_min,    mi_max)
    ASPLI_n = safe_norm(aspli, aspli_min, aspli_max)
    D = (EI_n + MI_n + ASPLI_n) / 3.0
    if D < 0.33:
        label = "easy"
    elif D < 0.66:
        label = "medium"
    else:
        label = "hard"
    return label, D

def main():
    pattern = '../cubicasa5k-3493-cubigraph/*/*/info.txt'
    info_files = glob.glob(pattern)

    # 1) First pass: compute raw indices
    file_indices = {}
    error_log_path = "./error.txt"
    open(error_log_path, 'w').close()

    for info_path in info_files:
        try:
            adj = read_adjacency_from_file(info_path)
            G   = build_graph(adj)
            file_indices[info_path] = compute_indices(G)
        except Exception as e:
            with open(error_log_path, 'a') as ef:
                ef.write(f"{os.path.dirname(info_path)} -> {e}\n")
            continue

    if not file_indices:
        print("No valid graphs found. Exiting.")
        return

    # 2) Determine global min/max
    all_EI    = [v[0] for v in file_indices.values()]
    all_MI    = [v[1] for v in file_indices.values()]
    all_ASPLI = [v[2] for v in file_indices.values()]
    EI_min, EI_max       = min(all_EI),    max(all_EI)
    MI_min, MI_max       = min(all_MI),    max(all_MI)
    ASPLI_min, ASPLI_max = min(all_ASPLI), max(all_ASPLI)

    # 3) Second pass: classify & store scores
    difficulty_map   = defaultdict(list)   # difficulty -> list of (path, score)
    for path, (ei, mi, aspli) in file_indices.items():
        diff, score = compute_difficulty(
            ei, mi, aspli,
            EI_min, EI_max,
            MI_min, MI_max,
            ASPLI_min, ASPLI_max
        )
        difficulty_map[diff].append((path, score))

        # append into each info.txt if not already present
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        if "Graph difficulty:" not in content:
            with open(path, 'a', encoding='utf-8') as f:
                f.write(f"\nRaw Indices: EI={ei}, MI={mi:.4f}, ASPLI={aspli:.4f}\n")
                f.write(f"score:  {score}\n")
                f.write(f"Graph difficulty: {diff}\n\n")

    # 4) Write summary with scores
    summary_file = "./graph_complexity_info_3493.txt"
    with open(summary_file, 'w') as sf:
        sf.write("Summary of all floorplans by difficulty:\n\n")
        for label in ["easy", "medium", "hard"]:
            entries = difficulty_map.get(label, [])
            sf.write(f"{label}: {len(entries)} file(s)\n")
            for path, score in entries:
                sf.write(f"   {path}  (score: {score:.6f})\n")
            sf.write("\n")

    print(f"Wrote summary of all floorplans by difficulty to {summary_file}")
    print(f"Any files that caused errors are logged in {error_log_path}")

if __name__ == "__main__":
    main()


Wrote summary of all floorplans by difficulty to ./graph_complexity_info_3493.txt
Any files that caused errors are logged in ./error.txt


In [6]:
import glob
import re
import ast
import os
import networkx as nx
from collections import defaultdict

def read_adjacency_from_file(path):
    with open(path, 'r') as f:
        content = f.read()
    match = re.search(r"Room Adjacency List:\s*(\{.*\})", content)
    if not match:
        raise ValueError(f"Could not find 'Room Adjacency List' in {path}.")
    return ast.literal_eval(match.group(1))

def build_graph(adjacency_dict):
    G = nx.Graph()
    for room, neighbors in adjacency_dict.items():
        for nbr, w in neighbors.items():
            G.add_edge(room, nbr, weight=w)
    return G

def largest_connected_subgraph(G):
    if nx.is_connected(G):
        return G
    largest_cc = max(nx.connected_components(G), key=len)
    return G.subgraph(largest_cc).copy()

def compute_indices(G):
    V = G.number_of_nodes()
    E = G.number_of_edges()
    EI = V + E
    MI = (E - V + 1) / (2.0 * V - 5) if (2 * V - 5) > 0 else 0.0
    H = largest_connected_subgraph(G)
    ASPLI = nx.average_shortest_path_length(H, weight='weight')
    return EI, MI, ASPLI

def compute_difficulty(ei, mi, aspli,
                       ei_min, ei_max,
                       mi_min, mi_max,
                       aspli_min, aspli_max):
    def safe_norm(x, xmin, xmax):
        return 0.0 if xmax == xmin else (x - xmin) / (xmax - xmin)
    EI_n    = safe_norm(ei,    ei_min,    ei_max)
    MI_n    = safe_norm(mi,    mi_min,    mi_max)
    ASPLI_n = safe_norm(aspli, aspli_min, aspli_max)
    D = (EI_n + MI_n + ASPLI_n) / 3.0
    if D < 0.33:
        label = "easy"
    elif D < 0.66:
        label = "medium"
    else:
        label = "hard"
    return label, D

def main():
    pattern = '../cubicasa5k-666/*/*/info.txt'
    info_files = glob.glob(pattern)

    # 1) First pass: compute raw indices
    file_indices = {}
    error_log_path = "./error.txt"
    open(error_log_path, 'w').close()

    for info_path in info_files:
        try:
            adj = read_adjacency_from_file(info_path)
            G   = build_graph(adj)
            file_indices[info_path] = compute_indices(G)
        except Exception as e:
            with open(error_log_path, 'a') as ef:
                ef.write(f"{os.path.dirname(info_path)} -> {e}\n")
            continue

    if not file_indices:
        print("No valid graphs found. Exiting.")
        return

    # 2) Determine global min/max
    all_EI    = [v[0] for v in file_indices.values()]
    all_MI    = [v[1] for v in file_indices.values()]
    all_ASPLI = [v[2] for v in file_indices.values()]
    EI_min, EI_max       = min(all_EI),    max(all_EI)
    MI_min, MI_max       = min(all_MI),    max(all_MI)
    ASPLI_min, ASPLI_max = min(all_ASPLI), max(all_ASPLI)

    # 3) Second pass: classify & store scores
    difficulty_map   = defaultdict(list)   # difficulty -> list of (path, score)
    for path, (ei, mi, aspli) in file_indices.items():
        diff, score = compute_difficulty(
            ei, mi, aspli,
            EI_min, EI_max,
            MI_min, MI_max,
            ASPLI_min, ASPLI_max
        )
        difficulty_map[diff].append((path, score))

        # append into each info.txt if not already present
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        if "Graph difficulty:" not in content:
            with open(path, 'a', encoding='utf-8') as f:
                f.write(f"\nRaw Indices: EI={ei}, MI={mi:.4f}, ASPLI={aspli:.4f}\n")
                f.write(f"score:  {score}\n")
                f.write(f"Graph difficulty: {diff}\n\n")

    # 4) Write summary with scores
    summary_file = "./graph_complexity_info_666.txt"
    with open(summary_file, 'w') as sf:
        sf.write("Summary of all floorplans by difficulty:\n\n")
        for label in ["easy", "medium", "hard"]:
            entries = difficulty_map.get(label, [])
            sf.write(f"{label}: {len(entries)} file(s)\n")
            for path, score in entries:
                sf.write(f"   {path}  (score: {score:.6f})\n")
            sf.write("\n")

    print(f"Wrote summary of all floorplans by difficulty to {summary_file}")
    print(f"Any files that caused errors are logged in {error_log_path}")

if __name__ == "__main__":
    main()


Wrote summary of all floorplans by difficulty to ./graph_complexity_info_666.txt
Any files that caused errors are logged in ./error.txt


In [4]:
import glob
import re
import ast
import os
import networkx as nx
from collections import defaultdict

def read_adjacency_from_file(path):
    with open(path, 'r') as f:
        content = f.read()
    match = re.search(r"Room Adjacency List:\s*(\{.*\})", content)
    if not match:
        raise ValueError(f"Could not find 'Room Adjacency List' in {path}.")
    return ast.literal_eval(match.group(1))

def build_graph(adjacency_dict):
    G = nx.Graph()
    for room, neighbors in adjacency_dict.items():
        for nbr, w in neighbors.items():
            G.add_edge(room, nbr, weight=w)
    return G

def largest_connected_subgraph(G):
    if nx.is_connected(G):
        return G
    largest_cc = max(nx.connected_components(G), key=len)
    return G.subgraph(largest_cc).copy()

def compute_indices(G):
    V = G.number_of_nodes()
    E = G.number_of_edges()
    EI = V + E
    MI = (E - V + 1) / (2.0 * V - 5) if (2 * V - 5) > 0 else 0.0
    H = largest_connected_subgraph(G)
    ASPLI = nx.average_shortest_path_length(H, weight='weight')
    return EI, MI, ASPLI

def compute_difficulty(ei, mi, aspli,
                       ei_min, ei_max,
                       mi_min, mi_max,
                       aspli_min, aspli_max):
    def safe_norm(x, xmin, xmax):
        return 0.0 if xmax == xmin else (x - xmin) / (xmax - xmin)
    EI_n    = safe_norm(ei,    ei_min,    ei_max)
    MI_n    = safe_norm(mi,    mi_min,    mi_max)
    ASPLI_n = safe_norm(aspli, aspli_min, aspli_max)
    D = (EI_n + MI_n + ASPLI_n) / 3.0
    if D < 0.33:
        label = "easy"
    elif D < 0.66:
        label = "medium"
    else:
        label = "hard"
    return label, D

def main():
    pattern = '../cubicasa5k-sample-69-with-hard/*/*/*/info.txt'
    info_files = glob.glob(pattern)

    # 1) First pass: compute raw indices
    file_indices = {}
    error_log_path = "./error.txt"
    open(error_log_path, 'w').close()

    for info_path in info_files:
        try:
            adj = read_adjacency_from_file(info_path)
            G   = build_graph(adj)
            file_indices[info_path] = compute_indices(G)
        except Exception as e:
            with open(error_log_path, 'a') as ef:
                ef.write(f"{os.path.dirname(info_path)} -> {e}\n")
            continue

    if not file_indices:
        print("No valid graphs found. Exiting.")
        return

    # 2) Determine global min/max
    all_EI    = [v[0] for v in file_indices.values()]
    all_MI    = [v[1] for v in file_indices.values()]
    all_ASPLI = [v[2] for v in file_indices.values()]
    EI_min, EI_max       = min(all_EI),    max(all_EI)
    MI_min, MI_max       = min(all_MI),    max(all_MI)
    ASPLI_min, ASPLI_max = min(all_ASPLI), max(all_ASPLI)

    # 3) Second pass: classify & store scores
    difficulty_map   = defaultdict(list)   # difficulty -> list of (path, score)
    for path, (ei, mi, aspli) in file_indices.items():
        diff, score = compute_difficulty(
            ei, mi, aspli,
            EI_min, EI_max,
            MI_min, MI_max,
            ASPLI_min, ASPLI_max
        )
        difficulty_map[diff].append((path, score))

        # append into each info.txt if not already present
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        if "Graph difficulty:" not in content:
            with open(path, 'a', encoding='utf-8') as f:
                f.write(f"\nRaw Indices: EI={ei}, MI={mi:.4f}, ASPLI={aspli:.4f}\n")
                f.write(f"score:  {score}\n")
                f.write(f"Graph difficulty: {diff}\n\n")

    # 4) Write summary with scores
    summary_file = "./graph_complexity_info_69.txt"
    with open(summary_file, 'w') as sf:
        sf.write("Summary of all floorplans by difficulty:\n\n")
        for label in ["easy", "medium", "hard"]:
            entries = difficulty_map.get(label, [])
            sf.write(f"{label}: {len(entries)} file(s)\n")
            for path, score in entries:
                sf.write(f"   {path}  (score: {score:.6f})\n")
            sf.write("\n")

    print(f"Wrote summary of all floorplans by difficulty to {summary_file}")
    print(f"Any files that caused errors are logged in {error_log_path}")

if __name__ == "__main__":
    main()


Wrote summary of all floorplans by difficulty to ./graph_complexity_info_69.txt
Any files that caused errors are logged in ./error.txt
